In [1]:
from pre_processing import *
from syllable_dictionary import *
from split_poem import *
import os
import numpy as np
from IPython.display import HTML

from HMM import unsupervised_HMM
from HMM_helper import (
    text_to_wordcloud,
    states_to_wordclouds,
    parse_observations,
    sample_sentence,
    visualize_sparsities,
    animate_emission
)
import random
from build_rhyme_dict import *
from syllable_dictionary import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
syllable_filename = "./project3/data/Syllable_dictionary.txt"
syllable_dict, syllable_end_dict = get_syllable_dict(syllable_filename)

In [3]:
rhyme_filename = "./project3/data/shakespeare.txt"
rhyme_dict = get_rhyme_dict(rhyme_filename)

In [4]:
filename = "./project3/data/shakespeare.txt"
obs_p, obs_map_p = pre_processing_sentences_reverse(filename)
hmm8_p = unsupervised_HMM(obs_p, 10, 100)
# generated_sentence = sample_sentence(hmm8_p, obs_map_p, n_words=300)

count of puncs =  0
0
10
20
30
40
50
60
70
80
90


In [5]:
def generate_rhyme(model, rhyme_dict, syllable_dict, syllable_end_dict, obs_map_p):
    poem = []
    non_dup = set()
    count = 0
    while count < 7:
        sentence1 = sample_sentence(model, obs_map_p, n_words=10)
        words1 = sentence1.split(" ")
        if (words1[0] in rhyme_dict) and (words1[0] not in non_dup):
            valid1, valid_sentence1 = exact_ten_syllables(words1, syllable_dict, syllable_end_dict)
            while valid1:
                sentence2 = sample_sentence(model, obs_map_p, n_words=10)
                words2 = sentence2.split(" ")
#                 print("========:", words1, words2)
                if (words2[0] in rhyme_dict[words1[0]]) and (words2[0] not in non_dup):
                    valid2, valid_sentence2 = exact_ten_syllables(words2, syllable_dict, syllable_end_dict)
                    if valid2:
                        poem.append(valid_sentence1)
                        poem.append(valid_sentence2)
                        non_dup.add(words1[0])
                        non_dup.add(words2[0])
                        count += 1
                        break
    
    for i in range(14):
        poem[i] = reverse(poem[i])
    
    poem_output = poem[0] +","+ poem[2] +","+ poem[1] +","+ poem[3] +","+ poem[4] +","+ poem[6] +","+ poem[5] +","+ poem[7] + ","+ \
                  poem[8] +","+ poem[10] +","+ poem[9] +","+ poem[11] +","+ poem[12] +","+ poem[13] +"."
    
    return poem_output

In [6]:
def reverse(sentence):
    words = sentence.split(" ")
    left = 0
    right = len(words)-1
    
    while left < right:
        temp = words[left]
        words[left] = words[right]
        words[right] = temp
        left += 1
        right -= 1
    
    sentence_reversed = " ".join(words)
    return sentence_reversed

In [7]:
def exact_ten_syllables(words, syllable_dict, syllable_end_dict):
    count = 0
    valid_words = []
    valid = False
    for word in words:
        curr_syllable_max = 0
        curr_syllable_min = 0
        
        if word in syllable_end_dict:
            if count + syllable_end_dict[word] == 10:
                valid_words.append(word)
                valid = True
                break
        
        if len(syllable_dict[word]) == 0:
            break
        elif len(syllable_dict[word]) == 1:
            curr_syllable_len = syllable_dict[word][0]
        else:
            print(len(syllable_dict[word]))
            rnd_idx = random.randint(0, len(syllable_dict[word])-1)
            curr_syllable_len = syllable_dict[word][rnd_idx]
        
        count += curr_syllable_len
        if count < 10:
            valid_words.append(word)
        elif count == 10:
                valid_words.append(word)
                valid = True
                break
        else:
            valid = False
    
    valid_sentence = ""
    if valid:
        for word in valid_words:
            valid_sentence += word + " "
    
    return valid, valid_sentence.strip()

In [8]:
# words = ["about","about","a-doting","about","a","about"]
# valid, valid_sentence = exact_ten_syllables(words, syllable_dict, syllable_end_dict)
# print(valid)
# print(valid_sentence)

In [9]:
generate_rhyme(hmm8_p, rhyme_dict, syllable_dict, syllable_end_dict, obs_map_p)

'thy if doth though thy breast and the thy stelled,have so alone rich doth my to thereby,so the cause a now there is common held,swear neglect i thy his riches doth die,it age permit worth remember sense me,where and self shall wait eyes they i thee rents,are i happier heart shall can win in thee,and have eye and am self-same ornaments,world and in my so thou of that cars eye,lack is lovely seasons to her the name,art thou my self is costly remedy,those though still day i summer fair till fame,to history thou with doth thralled lend,but by hope lives i thee my as doth tend.'

In [ ]:
print(rhyme_dict)